In [1]:
# OPEN AI
import os
import openai

from dotenv import load_dotenv

load_dotenv()

# Open AI
openai.api_key = os.getenv("OPENAI_API_KEY")
models = openai.Model.list()


# def get_embedding(text, model="text-embedding-ada-002"):
#     text = text.replace("\n", " ")
#     return openai.Embedding.create(input=[text], model=model)["data"][0]["embedding"]

In [3]:
from curses import meta
import chromadb
import os
import datetime
from chromadb.config import Settings
import docx


LOCAL_CHROMADB_DIR = "local_chroma_db"


def get_client():
    client = chromadb.Client(
        Settings(chroma_db_impl="duckdb+parquet", persist_directory=LOCAL_CHROMADB_DIR)
    )
    return client


def get_collection(client):
    journal_collection = client.get_or_create_collection("journal_entries")

    return journal_collection


def remove_extension(file_name):
    name_without_extension = os.path.splitext(file_name)[0]
    return name_without_extension


def read_docx(file_path):
    document = docx.Document(file_path)
    text = []
    for paragraph in document.paragraphs:
        text.append(paragraph.text)
    return "\n".join(text)


def get_documents_to_add(directory):
    documents = []
    metadatas = []
    ids = []
    count = 1
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)

            if file.startswith("."):
                continue

            # gather info from journals
            last_modified_date = str(
                datetime.datetime.fromtimestamp(os.path.getmtime(file_path))
            )
            title = remove_extension(file)
            text = read_docx(file_path)
            documents.append(f"{title} {text}")
            metadatas.append({"last_modified_date": last_modified_date})
            ids.append(f"Id{count}")
            count += 1

    return ids, documents, metadatas


def add_documents_to_collection(collection, ids, documents, metadatas):
    collection.add(ids=ids, documents=documents, metadatas=metadatas)


def first_time_batch_load():
    ids, documents, metadatas = get_documents_to_add("private-journals")
    add_documents_to_collection(collection, ids, documents, metadatas)


client = get_client()
collection = get_collection(client)
# first_time_batch_load()

In [2]:
from chromadb.utils import embedding_functions

basic_ef = embedding_functions.DefaultEmbeddingFunction()

prompt = "what would my past journals, what would you say about happiness"
query_embeddings = basic_ef([prompt])

documents = collection.query(
    query_embeddings=query_embeddings,
    n_results=1,
)["documents"]

closest_doc = documents[0][0]

# response = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {
#             "role": "system",
#             "content": "You are a kind and you are the the past self talking to the present self, based on journal entries provided by assistant. Be prophetic and cite examples in your responses from the journals.",
#         },
#         {"role": "assistant", "content": closest_doc},
#         {"role": "user", "content": prompt},
#     ],
# )
# response

NameError: name 'collection' is not defined

In [11]:
def db_is_empty(client: chromadb.Client) -> bool:
    return len(client.list_collections()) == 0

In [16]:
collection.count()

101

In [ ]:
s